In [ ]:
import numpy as np
from section4_1_Q import get_Q_T_H_d_t_i

## 第四節　FF暖房機

1．適用範囲～4．記号及び単位は省略

### 5．最大暖房出力

$$\large
Q_{max,H,d,t} = q_{max,H} \times 3600 \times 10^{-6}
$$

$Q_{max,H,d,t}$：日付$d$の時刻$t$における1時間当たりの最大暖房出力(MJ/h)  
$q_{max,H}$：最大暖房能力(W)

In [ ]:
def get_Q_max_H_d_t(q_max_H):
    return np.ones(24 * 365) * q_max_H * 3600 * 10 ** (-6)  # (1)

### 6．暖房エネルギー消費量

#### 6.1　消費電力量

$$
\large
E_{E,H,d,t} = \left\{ \begin{array}{ll}
    P_{rtd,H} \times \frac{Q_{T,H,d,t}}{Q_{max,H,d,t}} \times 10^{-3} & \small(Q_{T,H,d,t} \ge q_{min,H} \times 3600 \times 10^{-6}の場合)\\ 
    \left(P_{rtd,H} \times \frac{Q_{T,H,d,t}}{Q_{max,H,d,t}} + P_{itm,H} \right) \times 10^{-3} & \small(Q_{T,H,d,t} \lt q_{min,H} \times 3600 \times 10^{-6}の場合)\\ 
    \end{array}
\right.
$$

$E_{E,H,d,t}$：日付$d$の時刻$t$における1時間当たりの消費電力量(kWh/h)  
$q_{min,H}$：連続運転時最小能力(W)  
$P_{rtd,H}$：定格暖房消費電力(W)  
$P_{itm,H}$：断続時消費電力(W)  
$Q_{T,H,d,t}$：日付$d$の時刻$t$における1時間当たりの処理暖房負荷(MJ/h)  
$Q_{max,H,d,t}$：日付$d$の時刻$t$における1時間当たりの最大暖房出力(MJ/h)  

日付$d$の時刻$t$における1時間当たりの処理暖房負荷$Q_{T,H,d,t}$が 0 の場合、$E_{E,H,d,t} = 0$とする。

In [ ]:
def get_E_E_H_d_t(q_max_H, q_min_H, P_rtd_H, P_itm_H, L_H_d_t):
    # 最大暖房出力
    Q_max_H_d_t = get_Q_max_H_d_t(q_max_H)

    # 処理暖房負荷
    Q_T_H_d_t = get_Q_T_H_d_t_i(Q_max_H_d_t, L_H_d_t, None, '居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合に該当しない場合')

    # 消費電力量
    tmp1 = P_rtd_H * Q_T_H_d_t / Q_max_H_d_t * 10 ** (-3)
    tmp1[Q_T_H_d_t < q_min_H * 3600 * 10 ** (-6)] = 0.0

    tmp2 = (P_rtd_H * Q_T_H_d_t / Q_max_H_d_t + P_itm_H) * 10 ** (-3)
    tmp2[Q_T_H_d_t >= q_min_H * 3600 * 10 ** (-6)] = 0.0

    E_E_H_d_t = tmp1 + tmp2

    # ただし、Q_T_H_d_tが0の場合は0
    E_E_H_d_t[Q_T_H_d_t == 0] = 0

    return E_E_H_d_t

#### 6.2　ガス消費量

日付$d$の時刻$t$における１時間当たりのガス消費量$E_{G,H,d,t}$は、FF暖房機の燃料がガスの場合日付$d$の時刻$t$における１時間当たりの燃料消費量$E_{F,H,d,t}$に等しいとし、FF暖房機の燃料が灯油の場合0とする。

In [ ]:
def get_E_G_H_d_t(fuel, q_max_H, e_rtd_H, L_H_d_t):
    if fuel == 'G':
        return get_E_F_H_d_t(q_max_H, e_rtd_H, L_H_d_t)
    elif fuel == 'K':
        return np.zeros(24 * 365)
    else:
        raise ValueError(fuel)

#### 6.3　灯油消費量

日付$d$の時刻$t$における１時間当たりの灯油消費量$E_{K,H,d,t}$は、FF暖房機の燃料が灯油の場合日付$d$の時刻$t$における１時間当たりの燃料消費量$E_{F,H,d,t}$に等しいとし、FF暖房機の燃料がガスの場合0とする。

In [ ]:
def get_E_K_H_d_t(fuel, q_max_H, e_rtd_H, L_H_d_t):
    if fuel == 'K':
        return get_E_F_H_d_t(q_max_H, e_rtd_H, L_H_d_t)
    elif fuel == 'G':
        return np.zeros(24 * 365)
    else:
        raise ValueError(fuel)

#### 6.4　その他の燃料による一次エネルギー消費量

日付$d$の時刻$t$における１時間当たりのその他の燃料による一次エネルギー消費量$E_{M,H,d,t}$は、0とする。

In [ ]:
def get_E_M_H_d_t():
    return np.zeros(24 * 365)

### 7．燃料消費量

$$\large
E_{F,H,d,t} = \frac{Q_{T,H,d,t}}{e_{rtd,H}}
$$

$E_{F,H,d,t}$：日付$d$の時刻$t$における1時間当たりの燃料消費量(MJ/h)  
$Q_{T,H,d,t}$：日付$d$の時刻$t$における1時間当たりの処理暖房負荷(MJ/h)  
$e_{rtd,H}$：定格燃焼効率  

In [ ]:
def get_E_F_H_d_t(q_max_H, e_rtd_H, L_H_d_t):
    # 最大暖房出力
    Q_max_H_d_t = get_Q_max_H_d_t(q_max_H)

    # 処理暖房負荷
    Q_T_H_d_t = get_Q_T_H_d_t_i(Q_max_H_d_t, L_H_d_t, None, '居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合に該当しない場合')

    E_F_H_d_t = Q_T_H_d_t / e_rtd_H  # (3)

    return E_F_H_d_t


def get_Q_UT_H_d_t(q_max_H, L_H_d_t):
    # 最大暖房出力
    Q_max_H_d_t = get_Q_max_H_d_t(q_max_H)

    # 処理暖房負荷
    Q_T_H_d_t = get_Q_T_H_d_t_i(Q_max_H_d_t, L_H_d_t, None, '居室のみを暖房する方式でかつ主たる居室とその他の居室ともに温水暖房を設置する場合に該当しない場合')

    # 未処理暖房負荷
    Q_UT_H_d_t = L_H_d_t - Q_T_H_d_t

    return Q_UT_H_d_t

## 付録 A 機器の性能を表す仕様の決定方法

### A.1　記号及び単位

省略

### A.2　最大暖房能力

$$\large
q_{max,H} = 240.1 \times A_{HCZ}
$$

$q_{max,H}$：最大暖房能力(W)  
$A_{HCZ}$：FF 暖房機により暖房される暖冷房区画の床面積(m<sup>2</sup>)  

In [ ]:
def get_q_max_H(A_HCZ):
    return 240.1 * A_HCZ  # (1)

### A.3　連続運転時最小能力

連続運転時最小能力$q_{min,H}$が0を下回る場合は0とする。

$$\large
q_{min,H} = 0.4334 \times q_{max,H} - 540.1
$$

$q_{min,H}$：連続運転時最小能力(W)

In [ ]:
def get_q_min_H(q_max_H):
    q_min_H = 0.4334 * q_max_H - 540.1  # (2)
    return max(0.0, q_min_H)

### A.4　定格燃焼効率

定格燃焼効率$e_{trd,H}$は0.860（86.0%）とするか、ガスを燃料にするものについては JIS S 2122 に定められた測定方法による熱効率又は灯油を燃料にするものについては JIS S 3031 に定められた測定方法による空気を暖める方式の機器の熱効率を確認し、1000 分の 1 未満の端数を四捨五入し小数第三位までの値とする。

In [ ]:
def get_e_rtd_H_default():
    return 0.860


def get_e_rtd_H(e_rtd_H_raw):
    return round(e_rtd_H_raw * 1000) / 1000

### A.5　定格暖房消費電力

$$\large
P_{rtd,H} = 3.13 \times 10^{-3} \times q_{max,H}
$$

$P_{rtd,H}$：定格暖房消費電力(W)

In [ ]:
def get_P_rtd_H(q_max_H):
    return 3.13 / 1000 * q_max_H  # (3)

### A.6　断続時消費電力

断続時消費電力$P_{itm,H}$は、40.0Wとする。

In [ ]:
def get_P_itm_H():
    return 40.0

### A.7　複数のFF暖房機が設置される場合の仕様の決定方法

「主たる居室」に 2 台以上の暖房機が設置される場合、「その他の居室」に 2 台以上の暖房機が設置される場合又は複数の「その他の居室」においてそれぞれに暖房機が設置される場合は、「主たる居室」及び「その他の居室」それぞれにおいて、複数の機器の定格燃焼効率のうち最も小さい値を採用することとする。

In [ ]:
def get_e_rtd_from_multi_devices(e_rtd_list):
    return min(e_rtd_list)